In [1]:
import pandas as pd
import math
import os
import urllib.request

C:\Users\klysm\AppData\Local\Temp\ipykernel_12984\2454439438.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# URLs dos arquivos CSV
url1 = 'https://raw.githubusercontent.com/davsimoes/mcde-pds/main/res/emprestimos1.txt'
url2 = 'https://raw.githubusercontent.com/davsimoes/mcde-pds/main/res/emprestimos2.txt'
url3 = 'https://raw.githubusercontent.com/davsimoes/mcde-pds/main/res/emprestimos3.txt'

urls = [url1, url2, url3]
# Pasta para armazenar os arquivos baixados
pasta_dados = "data2"

In [4]:
# Função para baixar um arquivo CSV a partir de uma URL
def baixar_arquivo(url, nome_arquivo):
    urllib.request.urlretrieve(url, nome_arquivo)

In [5]:
# Verifica se a pasta de dados existe, caso contrário, cria-a
if not os.path.exists(pasta_dados):
    os.makedirs(pasta_dados)

In [8]:
# Baixa os arquivos CSV para a pasta de dados
for i, url in enumerate(urls):
    nome_arquivo = os.path.join(pasta_dados, f"file{i+1}.csv")
    baixar_arquivo(url, nome_arquivo)
    print(f"Arquivo {nome_arquivo} baixado com sucesso.")

Arquivo data2\file1.csv baixado com sucesso.
Arquivo data2\file2.csv baixado com sucesso.
Arquivo data2\file3.csv baixado com sucesso.


In [43]:
# Função para ler um arquivo CSV e retornar um dicionário de listas
def ler_csv_colunas(nome_arquivo):
    df = pd.read_csv(nome_arquivo)
    df.fillna(0, inplace=True)
    return df.to_dict(orient='list')

In [44]:
# Função para calcular os PMIs e adicionar ao dicionário de listas     
def emprestimo_pmi(montante, duracao, taxa, entrada=0):
    """Calcula a prestação mensal igual (PMI) para um empréstimo.
    
    Argumentos:
        montante - Montante total a aplicar (empréstimo + entrada)
        duracao - Duração do empréstimo (em meses)
        taxa - Taxa de juro (mensal)
        entrada (opcional) - Entrada opcional (deduzida do montante)
    """
    montante_emprestimo = montante - entrada
    try:
        pmi = montante_emprestimo * taxa * ((1+taxa)**duracao) / (((1+taxa)**duracao)-1)
    except ZeroDivisionError:
        pmi = montante_emprestimo / duracao
    pmi = math.ceil(pmi)
    return pmi

In [47]:
# Função para PMIs dicionário de listas
def calcular_pmis(dados):
    dados["pmi"] = []
    for i in range(len(dados["montante"])):
        montante = int(dados["montante"][i])
        duracao = int(dados["duracao"][i])
        taxa = float(dados["taxa"][i])
        entrada = int(dados["entrada"][i]) if dados["entrada"][i] else 0
        pmi = emprestimo_pmi(montante, duracao, taxa, entrada)
        dados["pmi"].append(pmi)

In [41]:
# Função para escrever os dados de um dicionário de listas em um arquivo CSV
def escrever_csv_colunas(dados, nome_arquivo):
    df = pd.DataFrame(dados)
    df.to_csv(nome_arquivo, index=False)

In [52]:
for i in range(len(urls)):
    print(f"Processando arquivo {i+1}...")
    
    nome_arquivo = os.path.join(pasta_dados, f"arquivo{i+1}.csv")
    print(f"Arquivo: {nome_arquivo}")

    dados = ler_csv_colunas(nome_arquivo)
    print (dados)

    calcular_pmis(dados)

    nome_arquivo_saida = os.path.join(pasta_dados, f"arquivo{i+1}_processado.csv")
    print(f"Arquivo de saída: {nome_arquivo_saida}")

    escrever_csv_colunas(dados, nome_arquivo_saida)

    print(f"Arquivo {nome_arquivo} processado e salvo em {nome_arquivo_saida}.")


Processando arquivo 1...
Arquivo: data2\arquivo1.csv
{'montante': [100000, 200000, 628400, 4637400, 42900, 916000, 45230, 991360, 423000], 'duracao': [36, 12, 120, 240, 90, 16, 48, 99, 27], 'taxa': [0.08, 0.1, 0.12, 0.06, 0.07, 0.13, 0.08, 0.08, 0.09], 'entrada': [20000.0, 0.0, 100000.0, 0.0, 8900.0, 0.0, 4300.0, 0.0, 47200.0]}
Arquivo de saída: data2\arquivo1_processado.csv
Arquivo data2\arquivo1.csv processado e salvo em data2\arquivo1_processado.csv.
Processando arquivo 2...
Arquivo: data2\arquivo2.csv
{'montante': [828400, 4633400, 42900, 983000, 15230], 'duracao': [120, 240, 90, 16, 48], 'taxa': [0.11, 0.06, 0.08, 0.14, 0.07], 'entrada': [100000.0, 0.0, 8900.0, 0.0, 4300.0]}
Arquivo de saída: data2\arquivo2_processado.csv
Arquivo data2\arquivo2.csv processado e salvo em data2\arquivo2_processado.csv.
Processando arquivo 3...
Arquivo: data2\arquivo3.csv
{'montante': [45230, 883000, 100000, 728400, 3637400, 82900, 316000, 15230, 991360, 323000, 4720010000, 528400, 8633400, 12900], '